In [ ]:
!pip install efficientnet_pytorch torchtoolbox

In [ ]:
# Imports here
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import pandas as pd
import os
import random
import math
import skimage.io
#from csv_loader import load_csv

# Tiff visualisation imports and downloads
import numpy as np
import tifffile as tiff

# For re-importing python modules
import importlib
#importlib.reload(csv_loader.py)

#for quadratic score calculator
from sklearn.metrics import cohen_kappa_score


In [ ]:
#use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [ ]:
# Creating ability to control how many pictures go into the training sample. For debugging / training purposes
sample_size = 10000
df = pd.read_csv('../input/prostate-cancer-grade-assessment/train.csv').copy().sample(sample_size)
df.to_csv("sample.csv", sep=",", index=False)

In [ ]:
class load_csv(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)# todo remove sample for debug
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
        
    
    def __getitem__(self, index):
        image_id = self.annotations.iloc[index, 0]
        img_path = os.path.join(self.root_dir, str(image_id) +".png")
        image = torch.from_numpy(skimage.io.imread(img_path)).permute(2,0,1).float()
        
        #y_label = torch.tensor(int(self.annotations.iloc[index,:]['isup_grade']))
        isup_grade = int(self.annotations.iloc[index,:]['isup_grade'])
        
        label = np.zeros(5).astype(np.float32)
        label[:isup_grade] = 1.
        
        
        self.transform= transforms.Compose([transforms.ToPILImage(),
                                            transforms.ToTensor()])
                                            
        if self.transform:
            image = self.transform(image)
        
        return (image, torch.tensor(label), image_id)

In [ ]:
# Loading csv dataset into the dataset loader function load_csv. 
dataset = load_csv(csv_file='sample.csv', root_dir='../input/prostate-cancer-tiles-4x4x128px-downsampling-4x/train_128x4x4_res1/train_128x4x4_res1')

# Creating sample subsets for validation and testing datasets
sample_size = dataset.annotations.shape[0]
train_ratio = .72
valid_ratio = .18
test_ratio = 1-(train_ratio + valid_ratio)
train_size = int(train_ratio*sample_size)
valid_size = int(valid_ratio*sample_size)
test_size = sample_size - train_size - valid_size

# Defining different datasets and respective dataloaders
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=5, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=5, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False)
entire_set_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

In [ ]:
# Creating model and uploading/creating needed training components
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=5)
model._fc = model._fc = nn.Sequential(nn.Linear(model._fc.in_features, 216),
                          nn.ReLU(),
                          nn.Linear(216, 36, bias=True),
                          nn.ReLU(),
                          nn.Linear(36, 5, bias=True)
#                          nn.Sigmoid()
                                     )


if torch.cuda.is_available():
    model = model.cuda()

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
testcriterion= nn.BCEWithLogitsLoss(reduction='none')
pred=torch.ones([10, 11], dtype=torch.float32)
truth=torch.ones([10, 11], dtype=torch.float32)#torch.full([10, 11], 1.0)
testcriterion(pred,truth)

In [ ]:
def validate_data_function(model, test_loader, criterion):
    test_loss = 0
    accuracy = 0
    output_list = []
    preds_list = []
    dec_list = []
    target_list = []
    loss_list = []
    
    with torch.no_grad():
        
        for ii, (inputs, labels, image_id) in enumerate(test_loader):
        
            inputs, labels = inputs.to(device), labels.to(device)
            output = model.forward(inputs)
        
            loss = criterion(output,labels)
               
            dec = output.sigmoid().sum(1).detach()
        
            output_list.append(output)
            preds_list.append(dec.round())
            target_list.append(labels.sum(1))
            dec_list.append(dec)
        
            loss_np = loss.detach().cpu().numpy()
            loss_list.append(loss_np)        
        test_loss = np.mean(loss_list)
        
        preds_list = torch.cat(preds_list).cpu().numpy()
        target_list = torch.cat(target_list).cpu().numpy()
        accuracy = np.mean(preds_list == target_list) * 100.
        
        #pred = output.cpu().data.numpy().argmax()
        #qwk = cohen_kappa_score(pred, labels, weights='quadratic')
    
    return test_loss, accuracy, image_id, preds_list, target_list

In [ ]:
# Training parameters and t=0 inputs
epochs = 3
print_every = 20
steps = 0
test_loss = 0

# May the training begin!
for epoch in range(epochs):
    model.train()
    running_loss = 0
    
    for ii, (inputs, labels, image_id) in enumerate(train_loader):
        steps += 1
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
       
        outputs = model.forward(inputs)#.forward(inputs)
        loss = criterion(outputs, labels)#.long())
        loss.backward()
        optimizer.step()
    
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()
            
            train_accuracy=0.0
            
            with torch.no_grad():
                valid_loss, accuracy, image_id,_,_ = validate_data_function(model, valid_loader, criterion)
                train_accuracy = (outputs.sum(1).round() == labels.sum(1)).float().mean()*100.
            
            print(f"Epoch: {epoch+1}/{epochs}..| "
                  f"Train loss: {running_loss/print_every:.3f}..| "
                  f"Train accuracy: {train_accuracy:.3f}..| "
                  f"Validation loss: {valid_loss/print_every:.3f}..| "                  
                  f"Validation accuracy: {accuracy:.3f}|"
                 )
            
            running_loss = 0
            model.train()
    
    path = 'base_model.pth'
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'classifier_state_dict': model._fc.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, path)
    
    
    model.cuda() # moving model to GPU for further training

In [ ]:
with torch.no_grad():
    valid_loss, accuracy, image_id = validate_data_function(model, valid_loader, criterion)

In [ ]:
image_id

In [ ]:
model.eval()
    
with torch.no_grad():
    test_loss, accuracy, image_id = validate_data_function(model, test_loader, criterion)
                
print("Test Accuracy: {}%".format(accuracy)